# Install & Import Dependencies
If you haven’t already installed the BigQuery client library, run this command in a Jupyter notebook cell:

In [2]:
!pip install google-cloud-bigquery


# Create a table with sensitive data in BigQuery

In [6]:
from google.cloud import bigquery

# Project, Location and Dataset ID
PROJECT_ID = "pii-iames1"  # Replace with your project ID
LOCATION = "us-central1"  # Replace with your location, e.g. us-central1
DATASET_ID = "sensitive_data"  
TABLE_ID = "sensitive_data"

# Initialize a BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# Define the schema for the table
schema = [
    bigquery.SchemaField("id", "INTEGER"),
    bigquery.SchemaField("full_name", "STRING"),
    bigquery.SchemaField("email", "STRING"),
    bigquery.SchemaField("phone_number", "STRING"),
    bigquery.SchemaField("f22_ceiling_height_ft", "INTEGER"),
    bigquery.SchemaField("f22_max_speed_mach", "FLOAT"),
    bigquery.SchemaField("f22_link16_frequency_mhz", "INTEGER"),
    bigquery.SchemaField("HAMMR_radar_range_miles", "INTEGER"),
    bigquery.SchemaField("classified_project_name", "STRING"),
    bigquery.SchemaField("controlled_technical_doc", "STRING"),
    bigquery.SchemaField("military_specification", "STRING"),
    bigquery.SchemaField("restricted_funding_source", "STRING"),
    bigquery.SchemaField("department", "STRING"),
    bigquery.SchemaField("project_code", "STRING"),
    bigquery.SchemaField("creation_date", "DATE"),
]

# Construct a full table reference
table_ref = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

# Create the table
table = bigquery.Table(table_ref, schema=schema)
table = client.create_table(table, exists_ok=True)

print(f"Table {table_ref} created or already exists.")

# Insert sample data into the table
rows_to_insert = [
    (1, "Alice Johnson", "alice.johnson@example.com", "123-456-7890", 72000, 2.25, 1210, 125, "Project Phantom", "TechSpec_Alpha", "MIL-STD-882E", "DARPA", "R&D", "PJT001", "2024-01-01"),
    (2, "Bob Smith", "bob.smith@example.com", "234-567-8901", 72000, 2.25, 1210, 125, "Project Shadow", "TechSpec_Beta", "MIL-STD-1472", "USAF", "Operations", "PJT002", "2024-01-02"),
    (3, "Charlie Davis", "charlie.davis@example.com", "345-678-9012", 72000, 2.25, 1210, 125, "Project Nightfall", "TechSpec_Gamma", "MIL-STD-810H", "NAVY", "Engineering", "PJT003", "2024-01-03"),
    (4, "Dana White", "dana.white@example.com", "456-789-0123", 72000, 2.25, 1210, 125, "Project Falcon", "TechSpec_Delta", "MIL-STD-461G", "ARPA", "Security", "PJT004", "2024-01-04"),
    (5, "Eve Green", "eve.green@example.com", "567-890-1234", 72000, 2.25, 1210, 125, "Project Storm", "TechSpec_Epsilon", "MIL-STD-464C", "DoD", "Logistics", "PJT005", "2024-01-05"),
    (6, "Frank Brown", "frank.brown@example.com", "678-901-2345", 72000, 2.25, 1210, 125, "Project Thunder", "TechSpec_Zeta", "MIL-STD-3023", "DARPA", "R&D", "PJT006", "2024-01-06"),
    (7, "Grace Hall", "grace.hall@example.com", "789-012-3456", 72000, 2.25, 1210, 125, "Project Eclipse", "TechSpec_Eta", "MIL-STD-889D", "USAF", "Operations", "PJT007", "2024-01-07"),
    (8, "Hank Lee", "hank.lee@example.com", "890-123-4567", 72000, 2.25, 1210, 125, "Project Titan", "TechSpec_Theta", "MIL-STD-461H", "NAVY", "Engineering", "PJT008", "2024-01-08"),
    (9, "Ivy Adams", "ivy.adams@example.com", "901-234-5678", 72000, 2.25, 1210, 125, "Project Vortex", "TechSpec_Iota", "MIL-STD-882E", "ARPA", "Security", "PJT009", "2024-01-09"),
    (10, "Jack Wilson", "jack.wilson@example.com", "012-345-6789", 72000, 2.25, 1210, 125, "Project Omega", "TechSpec_Kappa", "MIL-STD-1472", "DoD", "Logistics", "PJT010", "2024-01-10"),
]

# Insert the rows into the table
errors = client.insert_rows(table, rows_to_insert)

if errors:
    print("Errors occurred while inserting data:", errors)
else:
    print("Data successfully inserted.")


Table pii-iames1.sensitive_data.sensitive_data created or already exists.
Data successfully inserted.


# Python Code to Classify Data Using Gemini Flash
This script:
Reads data from sensitive_data in BigQuery.
Sends the data to Gemini Flash 1.5 for classification.
Writes the classification results into classified_data table in BigQuery.

In [8]:
from google.cloud import bigquery
import vertexai
from vertexai.generative_models import GenerativeModel, Part


# Define the table IDs
source_table_id = f"{PROJECT_ID}.{DATASET_ID}.sensitive_data"
destination_table_id = f"{PROJECT_ID}.{DATASET_ID}.classified_data"

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=LOCATION)

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

# Define the schema for the 'classified_data' table
schema = [
    bigquery.SchemaField("column_name", "STRING", mode="NULLABLE"),
    bigquery.SchemaField("classification", "STRING", mode="NULLABLE"),
]

def create_table(table_id, schema):
    """Creates or replaces a table in BigQuery."""
    table_ref = bigquery.Table(table_id, schema=schema)

    try:
        # Delete the existing table if it exists
        client.delete_table(table_id, not_found_ok=True)
        print(f"Deleted existing table {table_id}")

        # Create a new table
        client.create_table(table_ref)
        print(f"Created table {table_id}")
    except Exception as e:
        print(f"Table creation failed. Exception: {e}")
        return False

    return True

def fetch_sample_data(table_id, column_name):
    """Fetches the first two non-null values from a column in BigQuery."""
    query = f"""
    SELECT {column_name}
    FROM `{table_id}`
    WHERE {column_name} IS NOT NULL
    LIMIT 2
    """
    query_job = client.query(query)
    results = [row[column_name] for row in query_job.result()]
    return results


def classify_column(column_name, sample_values):
    """Classifies a column based on its name and sample values."""
    model = GenerativeModel("gemini-1.5-pro")
    prompt = f"""
    You are a data classification expert. Your task is to classify the following column based on its name and first two data values.
    Classify it as either:
    - "Export Controlled Information (ECI)"
    - "Personally Identifiable Information (PII)"
    - "NEITHER".

    Column Name: {column_name}
    Sample Values: {sample_values}

    Strictly return one of the three categories: "ECI", "PII", or "NEITHER".
    """

    try:
        response = model.generate_content(prompt)
        classification = response.text.strip().upper()

        # # Ensure valid classification
        # if classification not in ["ECI", "PII", "NEITHER"]:
        #     print(f"Invalid classification received: {classification}. Defaulting to NEITHER.")
        #     classification = "NEITHER"

        return classification

    except Exception as e:
        print(f"Error classifying column {column_name}: {e}")
        return "NEITHER"  # Default in case of error

    
def main():
    """Main function to classify columns based on name and sample values."""
    
    # Create or replace the destination table
    if not create_table(destination_table_id, schema):
        print("Exiting program since we cannot create table.")
        return

    # Get schema of source table
    table = client.get_table(source_table_id)
    source_schema = table.schema

    # Prepare rows to insert into the destination table
    rows_to_insert = []
    for field in source_schema:
        column_name = field.name
        sample_values = fetch_sample_data(source_table_id, column_name)
        classification = classify_column(column_name, sample_values)
        
        rows_to_insert.append({
            "column_name": column_name,
            "classification": classification
        })

    # Insert rows into BigQuery table
    table_ref = client.get_table(destination_table_id)
    errors = client.insert_rows_json(table_ref, rows_to_insert)


    if errors:
        print(f"Encountered errors while inserting rows: {errors}")
    else:
        print(f"Successfully classified columns and wrote to {destination_table_id}")

if __name__ == "__main__":
    main()

Deleted existing table pii-iames1.sensitive_data.classified_data
Created table pii-iames1.sensitive_data.classified_data
Successfully classified columns and wrote to pii-iames1.sensitive_data.classified_data


# View Classified Table in BigQuery
Go to BigQuery and view the classified_data table to view the classes